# 問題例の作成

In [1]:
class TW():
    def __init__(self, e, l):
        self.e = e
        self.l = l

In [2]:
class Coordinate():
    def __init__(self, x, y):
        self.x = x
        self.y = y

In [7]:
class piecewise_func():
    def __init__(self, xs, ys):
        self.xs = xs
        self.ys = ys

In [8]:
class Customer():
    def __init__(self, d, tw, coo, reward):
        self.demand = d
        self.tw = TW(tw[0], tw[1])
        self.coordinate = Coordinate(coo[0], coo[1])
        self.reward = reward

In [9]:
class Vehicle():
    def __init__(self, c, tour):
        self.capacity = c
        self.tour = tour

In [10]:
a = [10,20]
b = [0,*a]
b

[0, 10, 20]

In [ ]:
def ex():
    """
    V : 客集合
        N : 客数
        d : 需要
        tw : 時間枠
        (x, y) : 座標
        penalty : 時間枠を違反した場合にかかる、単位時間あたりの違反料
    K : 車両集合
        c : 容料
        #loading : 現在積んでいる荷物の量
        tour : 巡回路
    """
    import random
    random.seed(0)
    N = random.randint(10, 50)
    max_d = 10
    max_tw = 10
    max_co = 50
    min_penalty = 10
    max_penalty = 20
    max_reward = 20
    V = {}
    sum_d = 0
    V[0] = Customer(0, [0,max_tw], (0,0), 0, piecewise_func(*(0, max_tw), *(0,0)))
    for i in range(1, N+1):
        d = random.uniform(-max_d, max_d)
        tw = sorted([random.uniform(0, max_tw) for i in range(2)])
        x, y = random.uniform(-max_co, max_co), random.uniform(-max_co, max_co)
        penalty = random.uniform(min_penalty, max_penalty)
        reward_e, reward_l = sorted([random.uniform(0, max_reward) for i in range(2), reverse=True])
        if tw[0] != 0:
            xs = [0, tw[0]] + tw
            ys = [penalty, penalty, reward_e, reward_l]
        else:
            xs = tw
            ys = [reward_e, reward_l]
        if tw[1] != S:
            tw += [tw[1], S]
            ys += [penalty, penalty]
        reward = piecewise_func(xs, ys)
        V[i] = Customer(d, tw, (x, y), reward)
        sum_d += d*(d>0)
        
    K = {}
    min_cap = max_d/7
    max_cap = max_d/5
    sum_cap = 0
    i = 0
    while sum_cap<sum_d:
        c = random.uniform(min_cap, max_cap)
        K[i] = Vehicle(c, {})
        sum_cap += c
        i += 1
    return V, K

In [ ]:
def ex_small():
    """
    V : 客集合
        N : 客数
        d : 需要
        tw : 時間枠
        (x, y) : 座標
        penalty : 時間枠を違反した場合にかかる、単位時間あたりの違反料
    K : 車両集合
        c : 容料
        tour : 巡回路
    """
    import random
    random.seed(0)
    N = 5
    V = {}
    d_list = [0,1,-2,3,-4,5]
    tw_list = [(0,10),(0,10),(1,6),(2,8),(3,7),(4,7)]
    coo_list = [(0,0) for i in range(5+1)]#[(0,0),(0,2),(1,3),(2,6),(3,3),(4,10)]
    penalty_list = [0,10,10,10,20,20]
    linearfunc_list = [(0,0),(1,0),(2,0),(3,0),(4,0),(5,0)]
    V[0] = Customer(d_list[0], tw_list[0], coo_list[0], penalty_list[0], piecewise_func(*tw_list[0], *linearfunc_list[0]))
    for i in range(1,N+1):
        d = d_list[i]
        tw = tw_list[i]
        x, y = coo_list[i]
        penalty = penalty_list[i]
        reward = Reward_func(*linearfunc_list[i])#piecewise_func(*tw, *linearfunc_list[i])
        V[i] = Customer(d, tw, (x, y), penalty, reward)
        
    M = 3
    c_list = [2,4,6]
    K = {k:Vehicle(c_list[k], {}) for k in range(M)}
    K[0].tour = {0,1}
    K[1].tour = {0,2,3}
    K[2].tour = {0,4,5}
    
    return V, K

In [ ]:
V, K = ex()

In [ ]:
for i in V:
    print(i, end="\t")
    for condition in V[i].__dict__.values():
        print(condition, end="\n\t")
    print()

In [ ]:
sum(i.demand for i in V.values())

In [ ]:
sum(i.capacity for i in K.values())

# 問題を解く

## 巡回路を生成  
先生のソルバーを使う

## 巡回路を評価（各顧客の出発時刻を決める）  
どのくらい時間がかかるか調べる

In [ ]:
def distance(i, j, V):
    x0 = V[i].coordinate.x
    y0 = V[i].coordinate.y
    x1 = V[j].coordinate.x
    y1 = V[j].coordinate.y
    return ((x0-x1)**2 + (y0-y1)**2)**(1/2)

In [ ]:
import gurobipy as gp

# インスタンスの生成
m = gp.Model("LP_for_VRP_small")

# 定数を設定
V, K = ex_small()

# 変数を設定
"""
x_ik : 顧客iへ車両kが到着する時刻を表す変数
y_i : 車両が顧客iに到着する時刻が、顧客iの時間枠に対して、どの程度嬉しいかを表す変数
"""
x = {(i,k):m.addVar(vtype=gp.GRB.CONTINUOUS, name=f"x({i},{k})") for k in K for i in K[k].tour}
y = {i:m.addVar(vtype=gp.GRB.CONTINUOUS, name=f"y({i})") for i in V}

m.update()

In [ ]:
# 目的関数を設定
## 各顧客iの到着時刻x_ikと顧客iの時間枠を元に計算した違反料の和を最小化する
m.setObjective(gp.quicksum(y[i] for i in V), sense=gp.GRB.MAXIMIZE)

In [ ]:
# 制約条件を設定
## 任意の車両の巡回路内における、各顧客への到着時刻の先行、後行に関する制約
for k in K:
    for i_p in K[k].tour:
        for i_f in K[k].tour:
            if i_p != i_f:
                m.addConstr(x[i_p, k] + distance(i_p, i_f, V) <= x[i_f, k])
## 全ての車両はデポを出発し、サービス時間内に到着するという制約
for k in K:
    m.addConstr(0 <= x[0,k])

In [ ]:
# 最適化
m.optimize()
                
# 解の表示
if m.Status == gp.GRB.OPTIMAL:
    for k in K:
        for i in K[k].tour:
            print(f"車両{k}が顧客{i}に到着する時刻は、{x[i, k].X}")
    print("最適値 : ", m.ObjVal)